In [ ]:
import re
import os

from tqdm import tqdm, tnrange
import json
import pandas as pd
import os
from collections import namedtuple


from nltk.corpus import wordnet as wn
import copy
import numpy as np

import tensorflow.keras as K

import utils
import parsers
import models

training_file_path = '../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.data.xml'
gold_file_path =  '../resources/WSD_Evaluation_Framework/Training_Corpora/SemCor/semcor.gold.key.txt'

# training_file_path = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2007/semeval2007.data.xml'
# gold_file_path = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2007/semeval2007.gold.key.txt'

training_file_path_dev = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013/semeval2013.data.xml'
gold_file_path_dev = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2013/semeval2013.gold.key.txt'
fine_senses_vocab_path = '../resources/semcor.vocab.WordNet.json'
input_vocab_path = '../resources/semcor.input.vocab.json'
input_antivocab_path = '../resources/semcor.leftout.vocab.json'
embedding_size = 32
batch_size = 64
LEARNING_RATE = 0.01
N_EPOCHS = 10
PADDING_SIZE = 50
print_model = False

In [ ]:
#loading dict
senses = utils.json_vocab_reader(fine_senses_vocab_path)
inputs, antivocab = utils.json_vocab_reader(input_vocab_path, input_antivocab_path)
output_vocab = utils.vocab_merge(senses, inputs)
reverse_output_vocab =  dict((v, k) for k, v in output_vocab.items())

K.backend.clear_session()

In [ ]:
dir_ = "../resources/WSD_Evaluation_Framework/Evaluation_Datasets"
eval_datasets = [i for i in os.listdir(dir_) if i.startswith("se")]
eval_datasets

In [ ]:
# training_file_path = '../resources/WSD_Evaluation_Framework/Evaluation_Datasets/semeval2015/semeval2015.data.xml'


In [ ]:
# ##################
# # Model loading #
# #################
loaded_model = K.models.load_model('../resources/models/model_2019-09-06_22:48:47_+0200.h5')
loaded_model.load_weights('../resources/models/model_weights_2019-09-06_22:48:47_+0200.h5')


In [6]:
# loaded_model.compile(loss = 'categorical_crossentropy',
#               optimizer = K.optimizers.Adam(lr=0.0015, clipnorm=1., clipvalue=0.5),
#               metrics = ['acc', K.metrics.Precision()])

# ## PREDICTION
# predicted_one_hot = loaded_model.predict([padded(X_test_uni), padded(X_test_bi)])
# #de - one hot encode
# predicted = np.argmax(predicted_one_hot, axis=2)

In [7]:
import generatorPrototype
import generators

In [8]:
def eval_parser(path = training_file_path_dev, batch_size = 64):

    data_flow = parsers.TrainingParser(path)
    sentence_batch = []
    for batch_count, sentence in enumerate(data_flow.parse(), start = 1):
        sentence_batch.append(sentence)

        if int(batch_count)%int(batch_size)==0:
            yield sentence_batch
            sentence_batch = []

In [73]:
batch_size = 64

In [74]:
eval_generator = generatorPrototype.get(batch_size = batch_size,
                                training_file_path = training_file_path_dev,
                                antivocab = antivocab,
                                output_vocab = output_vocab,
                                PADDING_SIZE = PADDING_SIZE)

In [77]:
real_words = utils.eval_parser(path = training_file_path_dev, batch_size = batch_size)
c=0
for batch_x, candidate_synsets in eval_generator:
    print("-"*30)
    print("-"*30)
    c+=1
    print("batch : {}".format(c))
    batch_real_words = next(real_words)
    print("predicting")
    print("-"*30)
    predictions = loaded_model.predict_on_batch(batch_x)
    PADDING_SIZE = 50
    for idx_sentence, sentence in enumerate(predictions):
        print(idx_sentence)
        real_sentence = batch_real_words[idx_sentence]

        #split based on Model padding
        real_sentence, mfs_part = real_sentence[:50], real_sentence[51:]
        for idx, entry in enumerate(real_sentence):

            if entry.instance == True:
                if idx<PADDING_SIZE:
                    word_prob = sentence[idx]

                    current_candidate_synsets = candidate_synsets[idx_sentence][idx]
                    prob_dist_candidate_synset = word_prob[current_candidate_synsets]
                    current_synset = np.argmax(prob_dist_candidate_synset)

                    if current_synset>4:#change after deleting start stop
                        single_lemma_prediction = reverse_output_vocab[current_synset]
                        print(entry.id_, single_lemma_prediction)
                    else:
                        word = entry.lemma
                        print(entry.id_, MFS.retrieve_item(word))
                else:
                    word = entry.lemma
                    print(entry.id_, MFS.retrieve_item(word))

------------------------------
------------------------------
batch : 1
predicting
------------------------------
0
d000.s000.t000 wn:00031264n
d000.s000.t001 wn:05898568n
d000.s000.t002 wn:02626604v
1
d000.s001.t000 wn:14519366n
d000.s001.t001 wn:08308497n
d000.s001.t002 wn:04887701n
d000.s001.t003 wn:07234657n
d000.s001.t004 wn:05704694n
d000.s001.t005 wn:15164463n
d000.s001.t006 wn:04073208n
d000.s001.t007 wn:06470073n
d000.s001.t008 wn:00351638n
d000.s001.t009 wn:15203791n
d000.s001.t010 wn:05832745n
d000.s001.t011 wn:06303888n
2
d000.s002.t000 wn:06387980n
d000.s002.t001 wn:13790912n
d000.s002.t002 wn:01110274n
d000.s002.t003 wn:00696189v
d000.s002.t004 wn:05690269n
d000.s002.t005 wn:00415676n
3
d000.s003.t000 wn:07445265n
d000.s003.t001 wn:07148192n
d000.s003.t002 wn:07309599n
d000.s003.t003 wn:09466280n
d000.s003.t004 wn:09623038n
d000.s003.t005 wn:08761868n
d000.s003.t006 wn:15169873n
4
d000.s004.t000 wn:08408900n
d000.s004.t001 wn:06387980n
d000.s004.t002 wn:13384557n
d000.s00

StopIteration: 

In [23]:
class MFS(object):
    """
    WSD performed via Most Frequent Sense (MFS) which always returns the predominant sense of a lemma
    """

    def retrieve_item(lemma):
        """
        Retreives the MFS of a lemma from the nltk package if exists else word
        """
        synsets = wn.synsets(str(lemma))
        
        if len(synsets) == 0:
            print("fail")
            return lemma
        else:
            mfs = synsets[0]
            return utils.WordNet.from_synset(mfs)

    def predict(sentence):
        """
        param: sentence of lemmas
        return sentence MFS
        """
        return [MFS.retrieve_item(lemma) for lemma in sentence]

In [72]:
PADDING_SIZE = 50
for idx_sentence, sentence in enumerate(predictions):
    print(idx_sentence)
    real_sentence = batch_real_words[idx_sentence]
    
    #split based on Model padding
    real_sentence, mfs_part = real_sentence[:50], real_sentence[51:]
    for idx, entry in enumerate(real_sentence):
        
        if entry.instance == True:
            if idx<PADDING_SIZE:
                word_prob = sentence[idx]

                current_candidate_synsets = candidate_synsets[idx_sentence][idx]
                prob_dist_candidate_synset = word_prob[current_candidate_synsets]
                current_synset = np.argmax(prob_dist_candidate_synset)

                if current_synset>4:#change after deleting start stop
                    single_lemma_prediction = reverse_output_vocab[current_synset]
                    print(entry.id_, single_lemma_prediction)
                else:
                    word = entry.lemma
                    print(entry.id_, MFS.retrieve_item(word))
            else:
                word = entry.lemma
                print(entry.id_, MFS.retrieve_item(word))


0
d000.s000.t000 wn:00031264n
d000.s000.t001 wn:05898568n
d000.s000.t002 wn:01253060n
1
d000.s001.t000 wn:14519366n
d000.s001.t001 wn:08308497n
d000.s001.t002 wn:04887701n
d000.s001.t003 wn:07234657n
d000.s001.t004 wn:02604760v
d000.s001.t005 wn:15164463n
d000.s001.t006 wn:02604760v
d000.s001.t007 wn:06470073n
d000.s001.t008 wn:00351638n
d000.s001.t009 wn:15203791n
d000.s001.t010 wn:05832745n
d000.s001.t011 wn:01437963a
2
d000.s002.t000 wn:06387980n
d000.s002.t001 wn:13790912n
d000.s002.t002 wn:01110274n
d000.s002.t003 wn:14877234n
d000.s002.t004 wn:05690269n
d000.s002.t005 wn:00415676n
3
d000.s003.t000 wn:07445265n
d000.s003.t001 wn:07148192n
d000.s003.t002 wn:13296899n
d000.s003.t003 wn:00696189v
d000.s003.t004 wn:09623038n
d000.s003.t005 wn:08761868n
d000.s003.t006 wn:15169873n
4
d000.s004.t000 wn:08408900n
d000.s004.t001 wn:06387980n
d000.s004.t002 wn:13384557n
d000.s004.t003 wn:08168978n
d000.s004.t004 wn:13742573n
d000.s004.t005 wn:13488310n
d000.s004.t006 wn:01437963a
5
d000.s00